In [ ]:
import sys
sys.path.insert(0,'/home/nishadgothoskar/jax3dp3/jax3dp3/posecnn-pytorch/PoseCNN-PyTorch/')
import jax.numpy as jnp
import bayes3d as b
import trimesh
import os
import numpy as np
import trimesh
import jax


# --- creating the model dir from the working directory
model_dir = os.path.join(j.utils.get_assets_dir(), "ycb_video_models/models")
print(f"{model_dir} exists: {os.path.exists(model_dir)}")
model_names = j.ycb_loader.MODEL_NAMES
IDX = 13
name = model_names[IDX]
print(name)


bop_ycb_dir = os.path.join(j.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = j.ycb_loader.get_test_img('52', '1', bop_ycb_dir)
intrinsics = j.Intrinsics(
    height=rgbd.intrinsics.height,
    width=rgbd.intrinsics.width,
    fx=rgbd.intrinsics.fx, fy=rgbd.intrinsics.fx,
    cx=rgbd.intrinsics.width/2.0, cy=rgbd.intrinsics.height/2.0,
    near=0.001, far=1.0
)

In [ ]:
import bayes3d.posecnn_densefusion
densefusion = jax3dp3.posecnn_densefusion.DenseFusion()


In [ ]:

mesh_path = os.path.join(model_dir,name,"textured.obj")
print(mesh_path)
mesh = j.mesh.load_mesh(mesh_path)

In [ ]:
NUM_IMAGES_PER_ITER = 1
key = jax.random.PRNGKey(41)
object_poses = jax.vmap(lambda key: j.distributions.gaussian_vmf(key, 0.00001, 0.001))(
    jax.random.split(key, NUM_IMAGES_PER_ITER)
)
# object_poses = jnp.array([jnp.eye(4)])
# object_poses = object_poses.at[:,:3,3].set(jnp.array([0.0, 0.0, 1.0]))
object_poses = object_poses.at[:,:3,3].set(jnp.array([0.083, 0.08324493,  1.0084537 ]))

In [ ]:
all_data = j.kubric_interface.render_multiobject_parallel([mesh_path], object_poses[None,...],
                                                          intrinsics, scaling_factor=1.0, lighting=2.0) # multi img singleobj

rgbd = all_data[0]

In [ ]:
rgb = rgbd.rgb
# rgb[rgb[:,:,0] < 2.0,:] = 255.0
j.get_rgb_image(rgb)

In [ ]:
results = densefusion.get_densefusion_results(rgb, rgbd.depth, rgbd.intrinsics, scene_name="1")